In [3]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

We shall use the ImageDataGenerator class to feed in the training and validation data to the model. This class can also be used to generate augmented data.

To know more about ImageDataGenerator class, visit https://keras.io/preprocessing/image/#imagedatagenerator-class

In [6]:
train_datagenerator = ImageDataGenerator(rescale=1./255)
test_datagenerator = ImageDataGenerator(rescale=1./255)

train_datagenerator = train_datagenerator.flow_from_directory(
    'train',
    target_size=(64,64),
    batch_size=40,
    class_mode='binary')

test_datagenerator = test_datagenerator.flow_from_directory(
    'test',
    target_size=(64,64),
    batch_size=10,
    class_mode='binary')

Found 40 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Our model will have 3 Convolution2D layers. You can increse or decrease as per your needs.

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3),padding='same', activation='relu', input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D((2,2),2),
    
    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),     
     
    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2),2),   
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [4]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0

We shall use Adam optimizer with a learning rate of 0.001 (hyperparameter). We choose 'binary_crossentropy' loss as our model is a binary calssifier (i.e, we have only 2 classes)

In [8]:
model.compile(loss='binary_crossentropy',
             optimizer=tf.keras.optimizers.Adam(0.001),
             metrics=['accuracy'])

At the end of each epoch we can check if the model has reached the required accuracy and terminate the training.

In [9]:
DESIRED_ACCURACY = 0.85

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if((logs.get('acc')>DESIRED_ACCURACY) and (logs.get('val_acc')>DESIRED_ACCURACY )):
      print("\nReached 85% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

To know more about fit_generator visit https://keras.io/models/model/#fit_generator

In [11]:
model.fit_generator(
    train_datagenerator,
    epochs=10,
    validation_data = test_datagenerator,
    callbacks = [callbacks]
    )

Epoch 1/10


TypeError: unorderable types: NoneType() > float()

In [8]:
model.save('mymodel.h5')